In [2]:
library(feather)

In [3]:
X <- read_feather("../data/170605_preprocessing.feather")
X[1:5,1:5]

Warning message:
“Coercing int64 to double”

age,age_over_90,gender,bmi,bmi_missing
0.10731277,0,0,0.002855699,0
0.07349061,0,1,0.142778950,0
0.27824055,0,0,-0.099502083,0
0.08745643,0,0,0.181029175,0
0.19488550,0,0,-0.078058425,0


# Covariates

In [4]:
covariates <- glm(outcome ~
    + age + age_over_90 
    + gender 
    + bmi + bmi_missing 
    + mech_vent + CHF 
    + mdrd + creatinine_missing + apsiii 
    + micu + on_vaso
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(covariates)


Call:
glm(formula = outcome ~ +age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso, family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3091  -0.7563  -0.5631   0.8937   2.3308  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.15776    0.20069 -10.752  < 2e-16 ***
age                 1.61180    0.35901   4.489 7.14e-06 ***
age_over_90         1.20725    0.21518   5.610 2.02e-08 ***
gender             -0.02757    0.10470  -0.263  0.79226    
bmi                 0.17098    0.14316   1.194  0.23238    
bmi_missing         0.35909    0.18916   1.898  0.05765 .  
mech_vent           0.35991    0.11060   3.254  0.00114 ** 
CHF                 0.01881    0.11224   0.168  0.86689    
mdrd                0.02376    0.05734   0.414  0.67855    
creatinine_missing  0.07755    0.58275   0.133  0.89414    
apsi

In [5]:
minimal <- step(covariates, direction = 'back')
minimal$anova
summary(minimal)

Start:  AIC=2270.08
outcome ~ +age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso

                     Df Deviance    AIC
- creatinine_missing  1   2244.1 2268.1
- CHF                 1   2244.1 2268.1
- gender              1   2244.2 2268.2
- mdrd                1   2244.2 2268.2
- micu                1   2245.8 2269.8
<none>                    2244.1 2270.1
- bmi_missing         1   2247.6 2271.6
- bmi                 1   2250.5 2274.5
- mech_vent           1   2254.7 2278.7
- age                 1   2264.9 2288.9
- age_over_90         1   2274.9 2298.9
- on_vaso             1   2299.6 2323.6
- apsiii              1   2351.6 2375.6

Step:  AIC=2268.1
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + apsiii + micu + on_vaso

              Df Deviance    AIC
- CHF          1   2244.1 2266.1
- gender       1   2244.2 2266.2
- mdrd         1   2244.3 2266.3
- micu         1   

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2156,2244.078,2270.078
- creatinine_missing,1,0.01746393,2157,2244.096,2268.096
- CHF,1,0.02830228,2158,2244.124,2266.124
- gender,1,0.06528606,2159,2244.189,2264.189
- mdrd,1,0.15950625,2160,2244.349,2262.349
- micu,1,1.76824967,2161,2246.117,2262.117



Call:
glm(formula = outcome ~ age + age_over_90 + bmi + bmi_missing + 
    mech_vent + apsiii + on_vaso, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2906  -0.7552  -0.5650   0.8913   2.3606  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.98884    0.14247 -13.960  < 2e-16 ***
age          1.59960    0.34332   4.659 3.17e-06 ***
age_over_90  1.19794    0.20725   5.780 7.46e-09 ***
bmi          0.17458    0.14762   1.183  0.23696    
bmi_missing  0.36632    0.18882   1.940  0.05237 .  
mech_vent    0.34316    0.10846   3.164  0.00156 ** 
apsiii       0.60458    0.05685  10.634  < 2e-16 ***
on_vaso      0.93711    0.12440   7.533 4.96e-14 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2581.0  on 2168  degrees of freedom
Residual deviance: 2246.1  on 2161  degrees of freedom
AI

# LV

## Fully adjusted

In [7]:
model <- update(covariates, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day1_missing + fluid_day1
        + lv_hyper * fluid_day1
        + lv_mild * fluid_day1
        + lv_depressed * fluid_day1 
        + lv_severe * fluid_day1 
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day1_missing + fluid_day1 + lv_hyper:fluid_day1 + 
    lv_mild:fluid_day1 + lv_depressed:fluid_day1 + lv_severe:fluid_day1, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3961  -0.7500  -0.5519   0.8572   2.4520  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.249276   0.205975 -10.920  < 2e-16 ***
age                      1.677036   0.363767   4.610 4.02e-06 ***
age_over_90              1.228859   0.217722   5.644 1.66e-08 ***
gender                  -0.043368   0.106451  -0.407 0.683717    
bmi                      0.167613   0.131824   1.271 0.203554    
bmi_missing              0.360355   0.191140   1.885 0.059390 .  
m

Third-order

In [6]:
model <- update(covariates, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day1_missing + fluid_day1
        + lv_hyper * fluid_day1 * micu
        + lv_mild * fluid_day1 * micu
        + lv_depressed * fluid_day1 * micu
        + lv_severe * fluid_day1 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day1_missing + fluid_day1 + lv_hyper:fluid_day1 + 
    micu:lv_hyper + micu:fluid_day1 + lv_mild:fluid_day1 + micu:lv_mild + 
    lv_depressed:fluid_day1 + micu:lv_depressed + lv_severe:fluid_day1 + 
    micu:lv_severe + micu:lv_hyper:fluid_day1 + micu:lv_mild:fluid_day1 + 
    micu:lv_depressed:fluid_day1 + micu:lv_severe:fluid_day1, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4303  -0.7550  -0.5433   0.8547   2.4183  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -2.29132    0.22264 -10.292  < 2e-16 ***
age                           1.73597    0.36607   4.742 2.11e-06 ***
age_over_90                   1.24398   

In [9]:
model <- update(covariates, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day2_missing + fluid_day2
        + lv_hyper * fluid_day2
        + lv_mild * fluid_day2
        + lv_depressed * fluid_day2
        + lv_severe * fluid_day2
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day2_missing + fluid_day2 + lv_hyper:fluid_day2 + 
    lv_mild:fluid_day2 + lv_depressed:fluid_day2 + lv_severe:fluid_day2, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3464  -0.7529  -0.5477   0.8537   2.3586  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.28373    0.20778 -10.991  < 2e-16 ***
age                      1.68537    0.36471   4.621 3.82e-06 ***
age_over_90              1.22229    0.21837   5.597 2.18e-08 ***
gender                  -0.04421    0.10665  -0.415 0.678464    
bmi                      0.18556    0.16414   1.130 0.258270    
bmi_missing              0.38598    0.19094   2.021 0.043231 *  
mech_ven

Third order

In [12]:
model <- update(covariates, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day2_missing + fluid_day2
        + lv_hyper * fluid_day2 * micu
        + lv_mild * fluid_day2 * micu
        + lv_depressed * fluid_day2 * micu
        + lv_severe * fluid_day2 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day2_missing + fluid_day2 + lv_hyper:fluid_day2 + 
    micu:lv_hyper + micu:fluid_day2 + lv_mild:fluid_day2 + micu:lv_mild + 
    lv_depressed:fluid_day2 + micu:lv_depressed + lv_severe:fluid_day2 + 
    micu:lv_severe + micu:lv_hyper:fluid_day2 + micu:lv_mild:fluid_day2 + 
    micu:lv_depressed:fluid_day2 + micu:lv_severe:fluid_day2, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3154  -0.7502  -0.5424   0.8161   2.3351  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -2.31190    0.22423 -10.311  < 2e-16 ***
age                           1.75457    0.36705   4.780 1.75e-06 ***
age_over_90                   1.23607   

In [14]:
model <- update(covariates, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day3_missing + fluid_day3
        + lv_hyper * fluid_day3 
        + lv_mild * fluid_day3 
        + lv_depressed * fluid_day3 
        + lv_severe * fluid_day3 
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day3_missing + fluid_day3 + lv_hyper:fluid_day3 + 
    lv_mild:fluid_day3 + lv_depressed:fluid_day3 + lv_severe:fluid_day3, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4283  -0.7511  -0.5499   0.8498   2.4438  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.319835   0.211115 -10.988  < 2e-16 ***
age                      1.647076   0.364578   4.518 6.25e-06 ***
age_over_90              1.204521   0.218480   5.513 3.52e-08 ***
gender                  -0.046221   0.106613  -0.434 0.664623    
bmi                      0.202087   0.196768   1.027 0.304404    
bmi_missing              0.412263   0.191216   2.156 0.031083 *  
m

In [13]:
model <- update(covariates, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day3_missing + fluid_day3
        + lv_hyper * fluid_day3 * micu
        + lv_mild * fluid_day3 * micu
        + lv_depressed * fluid_day3 * micu
        + lv_severe * fluid_day3 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day3_missing + fluid_day3 + lv_hyper:fluid_day3 + 
    micu:lv_hyper + micu:fluid_day3 + lv_mild:fluid_day3 + micu:lv_mild + 
    lv_depressed:fluid_day3 + micu:lv_depressed + lv_severe:fluid_day3 + 
    micu:lv_severe + micu:lv_hyper:fluid_day3 + micu:lv_mild:fluid_day3 + 
    micu:lv_depressed:fluid_day3 + micu:lv_severe:fluid_day3, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3287  -0.7480  -0.5444   0.8425   2.4770  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -2.352452   0.227054 -10.361  < 2e-16 ***
age                           1.710569   0.367006   4.661 3.15e-06 ***
age_over_90                   1.23379

## Not adjusted

In [15]:
model <- glm(outcome ~ 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day1_missing + fluid_day1
        + lv_mild * fluid_day1 
        + lv_hyper * fluid_day1 
        + lv_mild * fluid_day1
        + lv_depressed * fluid_day1
        + lv_severe * fluid_day1
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ +lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day1_missing + fluid_day1 + lv_mild * fluid_day1 + 
    lv_hyper * fluid_day1 + lv_mild * fluid_day1 + lv_depressed * 
    fluid_day1 + lv_severe * fluid_day1, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6034  -0.8025  -0.7164   1.2949   2.0355  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -1.09750    0.06435 -17.054  < 2e-16 ***
lv_missing               0.13261    0.15424   0.860 0.389921    
lv_hyper                 0.59957    0.17650   3.397 0.000681 ***
lv_mild                 -0.14555    0.23595  -0.617 0.537316    
lv_depressed             0.53156    0.19580   2.715 0.006631 ** 
lv_severe                0.76371    0.18674   4.090 4.32e-05 ***
fluid_day1_missing      -0.19053    0.22307  -0.854 0.393021    
fluid_day1               0.26787    0

In [16]:
model <- glm(outcome ~ 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day2_missing + fluid_day2
        + lv_mild * fluid_day2
        + lv_hyper * fluid_day2 
        + lv_mild * fluid_day2
        + lv_depressed * fluid_day2
        + lv_severe * fluid_day2
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ +lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day2_missing + fluid_day2 + lv_mild * fluid_day2 + 
    lv_hyper * fluid_day2 + lv_mild * fluid_day2 + lv_depressed * 
    fluid_day2 + lv_severe * fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6601  -0.8001  -0.7082   1.2633   2.1774  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -1.11343    0.06665 -16.704  < 2e-16 ***
lv_missing               0.15746    0.15483   1.017 0.309189    
lv_hyper                 0.62222    0.17677   3.520 0.000431 ***
lv_mild                 -0.15196    0.23582  -0.644 0.519326    
lv_depressed             0.54668    0.19622   2.786 0.005335 ** 
lv_severe                0.79193    0.19036   4.160 3.18e-05 ***
fluid_day2_missing      -0.04397    0.14462  -0.304 0.761106    
fluid_day2               0.35302    0

In [17]:
model <- glm(outcome ~ 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day3_missing + fluid_day3
        + lv_mild * fluid_day3
        + lv_hyper * fluid_day3 
        + lv_mild * fluid_day3
        + lv_depressed * fluid_day3
        + lv_severe * fluid_day3
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ +lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day3_missing + fluid_day3 + lv_mild * fluid_day3 + 
    lv_hyper * fluid_day3 + lv_mild * fluid_day3 + lv_depressed * 
    fluid_day3 + lv_severe * fluid_day3, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4577  -0.7963  -0.7277   1.3084   2.0559  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -1.09371    0.07115 -15.371  < 2e-16 ***
lv_missing               0.16935    0.15494   1.093 0.274378    
lv_hyper                 0.65586    0.17347   3.781 0.000156 ***
lv_mild                 -0.15687    0.23741  -0.661 0.508779    
lv_depressed             0.55835    0.19686   2.836 0.004564 ** 
lv_severe                0.77428    0.18553   4.173 3.00e-05 ***
fluid_day3_missing      -0.09987    0.10813  -0.924 0.355662    
fluid_day3               0.38480    0

## Minimally adjusted

In [18]:
model <- update(minimal, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day1_missing + fluid_day1 
        + lv_mild * fluid_day1 * micu 
        + lv_hyper * fluid_day1 * micu
        + lv_mild * fluid_day1 * micu
        + lv_depressed * fluid_day1 * micu
        + lv_severe * fluid_day1 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + bmi + bmi_missing + 
    mech_vent + apsiii + on_vaso + lv_missing + lv_hyper + lv_mild + 
    lv_depressed + lv_severe + fluid_day1_missing + fluid_day1 + 
    micu + lv_mild:fluid_day1 + lv_mild:micu + fluid_day1:micu + 
    lv_hyper:fluid_day1 + lv_hyper:micu + lv_depressed:fluid_day1 + 
    lv_depressed:micu + lv_severe:fluid_day1 + lv_severe:micu + 
    lv_mild:fluid_day1:micu + lv_hyper:fluid_day1:micu + lv_depressed:fluid_day1:micu + 
    lv_severe:fluid_day1:micu, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4102  -0.7553  -0.5441   0.8426   2.4561  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -2.30157    0.21783 -10.566  < 2e-16 ***
age                           1.67300    0.35043   4.774 1.81e-06 ***
age_over_90                   1.20713    0.21082   5.726 1.03e-08 ***
bmi          

In [19]:
model <- update(minimal, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day2_missing + fluid_day2
        + lv_mild * fluid_day2 * micu 
        + lv_hyper * fluid_day2 * micu
        + lv_mild * fluid_day2 * micu
        + lv_depressed * fluid_day2 * micu
        + lv_severe * fluid_day2 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + bmi + bmi_missing + 
    mech_vent + apsiii + on_vaso + lv_missing + lv_hyper + lv_mild + 
    lv_depressed + lv_severe + fluid_day2_missing + fluid_day2 + 
    micu + lv_mild:fluid_day2 + lv_mild:micu + fluid_day2:micu + 
    lv_hyper:fluid_day2 + lv_hyper:micu + lv_depressed:fluid_day2 + 
    lv_depressed:micu + lv_severe:fluid_day2 + lv_severe:micu + 
    lv_mild:fluid_day2:micu + lv_hyper:fluid_day2:micu + lv_depressed:fluid_day2:micu + 
    lv_severe:fluid_day2:micu, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3004  -0.7494  -0.5449   0.8144   2.3478  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -2.32118    0.21946 -10.577  < 2e-16 ***
age                           1.69140    0.35119   4.816 1.46e-06 ***
age_over_90                   1.19817    0.21157   5.663 1.49e-08 ***
bmi          

In [20]:
model <- update(minimal, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day3_missing + fluid_day3
        + lv_mild * fluid_day3 * micu 
        + lv_hyper * fluid_day3 * micu
        + lv_mild * fluid_day3 * micu
        + lv_depressed * fluid_day3 * micu
        + lv_severe * fluid_day3 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + bmi + bmi_missing + 
    mech_vent + apsiii + on_vaso + lv_missing + lv_hyper + lv_mild + 
    lv_depressed + lv_severe + fluid_day3_missing + fluid_day3 + 
    micu + lv_mild:fluid_day3 + lv_mild:micu + fluid_day3:micu + 
    lv_hyper:fluid_day3 + lv_hyper:micu + lv_depressed:fluid_day3 + 
    lv_depressed:micu + lv_severe:fluid_day3 + lv_severe:micu + 
    lv_mild:fluid_day3:micu + lv_hyper:fluid_day3:micu + lv_depressed:fluid_day3:micu + 
    lv_severe:fluid_day3:micu, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3122  -0.7462  -0.5439   0.8333   2.5135  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -2.36085    0.22237 -10.617  < 2e-16 ***
age                           1.66759    0.35157   4.743 2.10e-06 ***
age_over_90                   1.20528    0.21174   5.692 1.25e-08 ***
bmi          

## Model selection via forward step-wise regression

In [27]:
model <- update(covariates, ~ .
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day1_missing + fluid_day1
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * lv_hyper * fluid_day1 + micu * lv_mild * fluid_day1
    + micu * lv_depressed * fluid_day1 + micu * lv_severe * fluid_day1
    ))
select <- step(model, direction = 'forward', scope = list(upper = scope))
select$anova
summary(select)

select2 <- update(select, ~ . + micu * lv_mild * fluid_day1)
summary(select2)

Start:  AIC=2262.19
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    fluid_day1_missing + fluid_day1

                          Df Deviance    AIC
+ micu:fluid_day1          1   2219.2 2261.2
<none>                         2222.2 2262.2
+ lv_severe:fluid_day1     1   2220.8 2262.8
+ micu:lv_severe           1   2221.6 2263.6
+ lv_depressed:fluid_day1  1   2221.9 2263.9
+ micu:lv_hyper            1   2222.1 2264.1
+ micu:lv_depressed        1   2222.1 2264.1
+ lv_hyper:fluid_day1      1   2222.1 2264.1
+ lv_mild:fluid_day1       1   2222.2 2264.2
+ micu:lv_mild             1   2222.2 2264.2

Step:  AIC=2261.2
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    fluid_day1_missing 

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2149,2222.185,2262.185
+ micu:fluid_day1,-1,2.988935,2148,2219.196,2261.196



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day1_missing + fluid_day1 + micu:fluid_day1, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4250  -0.7539  -0.5493   0.8582   2.4786  

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.2303914  0.2054956 -10.854  < 2e-16 ***
age                 1.6922152  0.3641140   4.647 3.36e-06 ***
age_over_90         1.2228062  0.2174205   5.624 1.86e-08 ***
gender             -0.0435185  0.1062858  -0.409 0.682211    
bmi                 0.1699862  0.1350485   1.259 0.208137    
bmi_missing         0.3629593  0.1910698   1.900 0.057484 .  
mech_vent           0.3708231  0.1125051   3.296 0.000981 ***
CHF                -0.0198165  0.1169708  -0.1


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day1_missing + fluid_day1 + micu:fluid_day1 + 
    micu:lv_mild + lv_mild:fluid_day1 + micu:lv_mild:fluid_day1, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4421  -0.7587  -0.5432   0.8469   2.4178  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.23604    0.20747 -10.777  < 2e-16 ***
age                      1.73633    0.36565   4.749 2.05e-06 ***
age_over_90              1.23126    0.21804   5.647 1.63e-08 ***
gender                  -0.04297    0.10651  -0.403  0.68660    
bmi                      0.17625    0.14610   1.206  0.22767    
bmi_missing              0.35773    0.19137   1.869  0.06159 .  
mech_vent           

In [28]:
model <- update(covariates, ~ .
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day2_missing + fluid_day2
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * lv_hyper * fluid_day2 + micu * lv_mild * fluid_day2
    + micu * lv_depressed * fluid_day2 + micu * lv_severe * fluid_day2
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * lv_mild * fluid_day2)
summary(select2)

Start:  AIC=2261.47
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    fluid_day2_missing + fluid_day2

                          Df Deviance    AIC
+ lv_severe:fluid_day2     1   2218.7 2260.7
+ lv_mild:fluid_day2       1   2218.8 2260.8
+ micu:fluid_day2          1   2219.3 2261.3
<none>                         2221.5 2261.5
+ lv_depressed:fluid_day2  1   2219.9 2261.9
+ micu:lv_severe           1   2220.9 2262.9
+ micu:lv_hyper            1   2221.3 2263.3
+ lv_hyper:fluid_day2      1   2221.3 2263.3
+ micu:lv_depressed        1   2221.4 2263.4
+ micu:lv_mild             1   2221.5 2263.5

Step:  AIC=2260.65
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    fluid_day2_missing

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2149,2221.469,2261.469
+ lv_severe:fluid_day2,-1,2.815893,2148,2218.653,2260.653
+ lv_mild:fluid_day2,-1,2.403386,2147,2216.250,2260.250
+ micu:fluid_day2,-1,2.047402,2146,2214.202,2260.202



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day2_missing + fluid_day2 + lv_severe:fluid_day2 + 
    lv_mild:fluid_day2 + micu:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3409  -0.7496  -0.5499   0.8416   2.3322  

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)          -2.24294    0.20746 -10.811  < 2e-16 ***
age                   1.68035    0.36444   4.611 4.01e-06 ***
age_over_90           1.21194    0.21808   5.557 2.74e-08 ***
gender               -0.04203    0.10661  -0.394  0.69338    
bmi                   0.18885    0.16912   1.117  0.26414    
bmi_missing           0.38429    0.19050   2.017  0.04367 *  
mech_vent             0.37394    0.11368   3.289  0.00100 *


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day2_missing + fluid_day2 + lv_severe:fluid_day2 + 
    lv_mild:fluid_day2 + micu:fluid_day2 + micu:lv_mild + micu:lv_mild:fluid_day2, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3536  -0.7483  -0.5467   0.8137   2.3199  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.24661    0.20914 -10.742  < 2e-16 ***
age                      1.73242    0.36632   4.729 2.25e-06 ***
age_over_90              1.21501    0.21885   5.552 2.83e-08 ***
gender                  -0.04477    0.10688  -0.419  0.67529    
bmi                      0.19869    0.19144   1.038  0.29934    
bmi_missing              0.38180    0.19077   2.001  0.04536 *

In [29]:
model <- update(covariates, ~ .
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day3_missing + fluid_day3
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * lv_hyper * fluid_day3 + micu * lv_mild * fluid_day3
    + micu * lv_depressed * fluid_day3 + micu * lv_severe * fluid_day3
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * lv_mild * fluid_day2)
summary(select2)

Start:  AIC=2259.57
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    fluid_day3_missing + fluid_day3

                          Df Deviance    AIC
+ micu:fluid_day3          1   2215.0 2257.0
+ lv_mild:fluid_day3       1   2216.1 2258.1
+ lv_depressed:fluid_day3  1   2217.0 2259.0
<none>                         2219.6 2259.6
+ micu:lv_severe           1   2219.1 2261.1
+ lv_severe:fluid_day3     1   2219.2 2261.2
+ micu:lv_hyper            1   2219.4 2261.4
+ micu:lv_depressed        1   2219.6 2261.6
+ micu:lv_mild             1   2219.6 2261.6
+ lv_hyper:fluid_day3      1   2219.6 2261.6

Step:  AIC=2257.01
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    fluid_day3_missing

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2149,2219.568,2259.568
+ micu:fluid_day3,-1,4.555316,2148,2215.013,2257.013
+ lv_mild:fluid_day3,-1,3.758530,2147,2211.254,2255.254
+ lv_depressed:fluid_day3,-1,2.015157,2146,2209.239,2255.239



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day3_missing + fluid_day3 + micu:fluid_day3 + 
    lv_mild:fluid_day3 + lv_depressed:fluid_day3, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3165  -0.7497  -0.5484   0.8511   2.4486  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.28492    0.21056 -10.851  < 2e-16 ***
age                      1.65707    0.36484   4.542 5.58e-06 ***
age_over_90              1.20998    0.21861   5.535 3.11e-08 ***
gender                  -0.05225    0.10668  -0.490  0.62426    
bmi                      0.20322    0.19744   1.029  0.30334    
bmi_missing              0.42026    0.19104   2.200  0.02782 *  
mech_vent                0.39477   


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day3_missing + fluid_day3 + fluid_day2 + 
    micu:fluid_day3 + lv_mild:fluid_day3 + lv_depressed:fluid_day3 + 
    micu:lv_mild + micu:fluid_day2 + lv_mild:fluid_day2 + micu:lv_mild:fluid_day2, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3375  -0.7464  -0.5435   0.8382   2.3302  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.290767   0.212927 -10.758  < 2e-16 ***
age                      1.711667   0.367069   4.663 3.12e-06 ***
age_over_90              1.218574   0.219964   5.540 3.03e-08 ***
gender                  -0.053169   0.106992  -0.497 0.619228    
bmi                      0.212886   0.222698   0.956 0.339103 

# plm_htn

## Fully adjusted

In [30]:
model <- update(covariates, ~ . 
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day1_missing + fluid_day1
        + plm_htn_abnormal * fluid_day1 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day1_missing + 
    fluid_day1 + plm_htn_abnormal:fluid_day1 + micu:plm_htn_abnormal + 
    micu:fluid_day1 + micu:plm_htn_abnormal:fluid_day1, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.5087  -0.7539  -0.5497   0.8478   2.4249  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -2.36792    0.26650  -8.885  < 2e-16 ***
age                               1.45765    0.36780   3.963 7.40e-05 ***
age_over_90                       1.09263    0.22001   4.966 6.82e-07 ***
gender                           -0.06823    0.10593  -0.644 0.519549    
bmi                               0.16807    0.13177   1.275 0.202139    
bmi_missing                 

In [38]:
model <- update(covariates, ~ . 
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day2_missing + fluid_day2
        + plm_htn_abnormal * fluid_day2 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day2_missing + 
    fluid_day2 + plm_htn_abnormal:fluid_day2 + micu:plm_htn_abnormal + 
    micu:fluid_day2 + micu:plm_htn_abnormal:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4898  -0.7489  -0.5507   0.8461   2.4098  

Coefficients:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -2.3703869  0.2678826  -8.849  < 2e-16 ***
age                               1.4038392  0.3673116   3.822 0.000132 ***
age_over_90                       1.0531649  0.2199967   4.787 1.69e-06 ***
gender                           -0.0720352  0.1059548  -0.680 0.496589    
bmi                               0.1853373  0.1630216   1.137 0.255585    
bmi_missing     

In [32]:
model <- update(covariates, ~ . 
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day3_missing + fluid_day3
        + plm_htn_abnormal * fluid_day3 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day3_missing + 
    fluid_day3 + plm_htn_abnormal:fluid_day3 + micu:plm_htn_abnormal + 
    micu:fluid_day3 + micu:plm_htn_abnormal:fluid_day3, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4969  -0.7541  -0.5477   0.8371   2.3966  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -2.39875    0.26916  -8.912  < 2e-16 ***
age                               1.34390    0.36709   3.661 0.000251 ***
age_over_90                       1.03512    0.22015   4.702 2.58e-06 ***
gender                           -0.07405    0.10604  -0.698 0.484975    
bmi                               0.20206    0.19358   1.044 0.296570    
bmi_missing                 

## Unadjusted

In [33]:
model <- glm(outcome ~ 
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day1_missing + fluid_day1
        + plm_htn_abnormal * fluid_day1 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ +plm_htn_missing + plm_htn_abnormal + 
    fluid_day1_missing + fluid_day1 + plm_htn_abnormal * fluid_day1 * 
    micu, family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2875  -0.8221  -0.7285   1.3193   2.0940  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -1.56313    0.21281  -7.345 2.06e-13 ***
plm_htn_missing                   0.17639    0.16037   1.100   0.2714    
plm_htn_abnormal                  0.57070    0.27920   2.044   0.0409 *  
fluid_day1_missing               -0.18002    0.22161  -0.812   0.4166    
fluid_day1                        0.21932    0.16263   1.349   0.1775    
micu                              0.37571    0.19198   1.957   0.0503 .  
plm_htn_abnormal:fluid_day1      -0.10126    0.24783  -0.409   0.6828    
plm_htn_abnormal:micu            -0.11437    0.27322  -0.419   0.6755    
fluid_day

In [34]:
model <- glm(outcome ~ 
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day2_missing + fluid_day2
        + plm_htn_abnormal * fluid_day2 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ +plm_htn_missing + plm_htn_abnormal + 
    fluid_day2_missing + fluid_day2 + plm_htn_abnormal * fluid_day2 * 
    micu, family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2357  -0.8271  -0.7118   1.3232   2.1149  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -1.57588    0.21546  -7.314 2.59e-13 ***
plm_htn_missing                   0.16465    0.16071   1.025   0.3056    
plm_htn_abnormal                  0.55385    0.28353   1.953   0.0508 .  
fluid_day2_missing               -0.05181    0.14408  -0.360   0.7192    
fluid_day2                        0.29321    0.17824   1.645   0.1000 .  
micu                              0.38403    0.19491   1.970   0.0488 *  
plm_htn_abnormal:fluid_day2      -0.05261    0.26272  -0.200   0.8413    
plm_htn_abnormal:micu            -0.09036    0.27783  -0.325   0.7450    
fluid_day

In [35]:
model <- glm(outcome ~ 
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day3_missing + fluid_day3
        + plm_htn_abnormal * fluid_day3 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ +plm_htn_missing + plm_htn_abnormal + 
    fluid_day3_missing + fluid_day3 + plm_htn_abnormal * fluid_day3 * 
    micu, family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2422  -0.8558  -0.7080   1.2847   1.9767  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -1.53050    0.21244  -7.204 5.83e-13 ***
plm_htn_missing                   0.17573    0.16068   1.094   0.2741    
plm_htn_abnormal                  0.55242    0.27789   1.988   0.0468 *  
fluid_day3_missing               -0.09508    0.10754  -0.884   0.3766    
fluid_day3                        0.07657    0.19000   0.403   0.6869    
micu                              0.34898    0.19028   1.834   0.0666 .  
plm_htn_abnormal:fluid_day3       0.08604    0.28510   0.302   0.7628    
plm_htn_abnormal:micu            -0.07958    0.27204  -0.293   0.7699    
fluid_day

## Model selection

In [39]:
model <- update(covariates, ~ .
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day1_missing + fluid_day1
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * plm_htn_abnormal * fluid_day1
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * plm_htn_abnormal * fluid_day1)
summary(select2)

Start:  AIC=2263.97
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_missing + plm_htn_abnormal + fluid_day1_missing + 
    fluid_day1

                              Df Deviance    AIC
+ micu:fluid_day1              1   2226.4 2262.4
+ plm_htn_abnormal:fluid_day1  1   2227.9 2263.9
<none>                             2230.0 2264.0
+ micu:plm_htn_abnormal        1   2229.7 2265.7

Step:  AIC=2262.42
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_missing + plm_htn_abnormal + fluid_day1_missing + 
    fluid_day1 + micu:fluid_day1

                              Df Deviance    AIC
<none>                             2226.4 2262.4
+ plm_htn_abnormal:fluid_day1  1   2224.8 2262.8
+ micu:plm_htn_abnormal        1   2226.1 2264.1


Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2152,2229.972,2263.972
+ micu:fluid_day1,-1,3.550394,2151,2226.421,2262.421



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day1_missing + 
    fluid_day1 + micu:fluid_day1, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4208  -0.7551  -0.5473   0.8596   2.3810  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.29391    0.23773  -9.649  < 2e-16 ***
age                 1.44231    0.36682   3.932 8.43e-05 ***
age_over_90         1.09244    0.21957   4.975 6.51e-07 ***
gender             -0.06310    0.10571  -0.597 0.550566    
bmi                 0.16998    0.13539   1.255 0.209305    
bmi_missing         0.39173    0.19218   2.038 0.041513 *  
mech_vent           0.38461    0.11244   3.421 0.000625 ***
CHF                -0.02412    0.11359  -0.212 0.831850    
mdrd                0.0337


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day1_missing + 
    fluid_day1 + micu:fluid_day1 + micu:plm_htn_abnormal + plm_htn_abnormal:fluid_day1 + 
    micu:plm_htn_abnormal:fluid_day1, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.5087  -0.7539  -0.5497   0.8478   2.4249  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -2.36792    0.26650  -8.885  < 2e-16 ***
age                               1.45765    0.36780   3.963 7.40e-05 ***
age_over_90                       1.09263    0.22001   4.966 6.82e-07 ***
gender                           -0.06823    0.10593  -0.644 0.519549    
bmi                               0.16807    0.13177   1.275 0.202139    
bmi_missing                 

In [40]:
model <- update(covariates, ~ .
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day2_missing + fluid_day2
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * plm_htn_abnormal * fluid_day2
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * plm_htn_abnormal * fluid_day2)
summary(select2)

Start:  AIC=2263.01
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_missing + plm_htn_abnormal + fluid_day2_missing + 
    fluid_day2

                              Df Deviance    AIC
+ micu:fluid_day2              1   2226.4 2262.4
+ plm_htn_abnormal:fluid_day2  1   2226.9 2262.9
<none>                             2229.0 2263.0
+ micu:plm_htn_abnormal        1   2228.8 2264.8

Step:  AIC=2262.42
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_missing + plm_htn_abnormal + fluid_day2_missing + 
    fluid_day2 + micu:fluid_day2

                              Df Deviance    AIC
<none>                             2226.4 2262.4
+ plm_htn_abnormal:fluid_day2  1   2224.6 2262.6
+ micu:plm_htn_abnormal        1   2226.2 2264.2


Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2152,2229.013,2263.013
+ micu:fluid_day2,-1,2.589993,2151,2226.423,2262.423



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day2_missing + 
    fluid_day2 + micu:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4203  -0.7533  -0.5508   0.8428   2.3790  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.31555    0.23972  -9.659  < 2e-16 ***
age                 1.39889    0.36647   3.817 0.000135 ***
age_over_90         1.05894    0.21966   4.821 1.43e-06 ***
gender             -0.06954    0.10580  -0.657 0.511006    
bmi                 0.18864    0.16762   1.125 0.260399    
bmi_missing         0.43430    0.19164   2.266 0.023435 *  
mech_vent           0.38983    0.11339   3.438 0.000586 ***
CHF                -0.00289    0.11362  -0.025 0.979711    
mdrd                0.0358


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day2_missing + 
    fluid_day2 + micu:fluid_day2 + micu:plm_htn_abnormal + plm_htn_abnormal:fluid_day2 + 
    micu:plm_htn_abnormal:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4898  -0.7489  -0.5507   0.8461   2.4098  

Coefficients:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -2.3703869  0.2678826  -8.849  < 2e-16 ***
age                               1.4038392  0.3673116   3.822 0.000132 ***
age_over_90                       1.0531649  0.2199967   4.787 1.69e-06 ***
gender                           -0.0720352  0.1059548  -0.680 0.496589    
bmi                               0.1853373  0.1630216   1.137 0.255585    
bmi_missing     

In [41]:
model <- update(covariates, ~ .
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day3_missing + fluid_day3
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * plm_htn_abnormal * fluid_day3
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * plm_htn_abnormal * fluid_day3)
summary(select2)

Start:  AIC=2260.32
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_missing + plm_htn_abnormal + fluid_day3_missing + 
    fluid_day3

                              Df Deviance    AIC
+ micu:fluid_day3              1   2221.5 2257.5
+ plm_htn_abnormal:fluid_day3  1   2223.9 2259.9
<none>                             2226.3 2260.3
+ micu:plm_htn_abnormal        1   2226.1 2262.1

Step:  AIC=2257.49
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_missing + plm_htn_abnormal + fluid_day3_missing + 
    fluid_day3 + micu:fluid_day3

                              Df Deviance    AIC
+ plm_htn_abnormal:fluid_day3  1   2219.4 2257.4
<none>                             2221.5 2257.5
+ micu:plm_htn_abnormal        1   2221.3 2259.3

Step:  AIC=2257.45
outcome ~ age + age_over_90 + gender + bmi + bmi_

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2152,2226.323,2260.323
+ micu:fluid_day3,-1,4.834336,2151,2221.489,2257.489
+ plm_htn_abnormal:fluid_day3,-1,2.040925,2150,2219.448,2257.448



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day3_missing + 
    fluid_day3 + micu:fluid_day3 + plm_htn_abnormal:fluid_day3, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.5017  -0.7522  -0.5462   0.8466   2.3758  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -2.34613    0.24218  -9.687  < 2e-16 ***
age                          1.34983    0.36677   3.680 0.000233 ***
age_over_90                  1.03339    0.22016   4.694 2.68e-06 ***
gender                      -0.07333    0.10604  -0.692 0.489222    
bmi                          0.20284    0.19480   1.041 0.297755    
bmi_missing                  0.45941    0.19233   2.389 0.016913 *  
mech_vent                    0.41239    0.11539   3.5


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day3_missing + 
    fluid_day3 + micu:fluid_day3 + plm_htn_abnormal:fluid_day3 + 
    micu:plm_htn_abnormal + micu:plm_htn_abnormal:fluid_day3, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4969  -0.7541  -0.5477   0.8371   2.3966  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -2.39875    0.26916  -8.912  < 2e-16 ***
age                               1.34390    0.36709   3.661 0.000251 ***
age_over_90                       1.03512    0.22015   4.702 2.58e-06 ***
gender                           -0.07405    0.10604  -0.698 0.484975    
bmi                               0.20206    0.19358   1.044 0.296570    
bmi_missing                 

# RV cavity

## Fully adjusted

In [43]:
model <- update(covariates, ~ . 
        + rvcav_missing + rvcav_dilated
        + fluid_day1_missing + fluid_day1
        + rvcav_dilated * fluid_day1 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day1_missing + 
    fluid_day1 + rvcav_dilated:fluid_day1 + micu:rvcav_dilated + 
    micu:fluid_day1 + micu:rvcav_dilated:fluid_day1, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3068  -0.7563  -0.5566   0.8612   2.3715  

Coefficients:
                               Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -2.108006   0.208610 -10.105  < 2e-16 ***
age                            1.663051   0.362311   4.590 4.43e-06 ***
age_over_90                    1.214069   0.216175   5.616 1.95e-08 ***
gender                        -0.035949   0.105205  -0.342 0.732574    
bmi                            0.160800   0.124419   1.292 0.196214    
bmi_missing                    0.346196   0.190295   1

In [44]:
model <- update(covariates, ~ . 
        + rvcav_missing + rvcav_dilated
        + fluid_day2_missing + fluid_day2
        + rvcav_dilated * fluid_day2 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day2_missing + 
    fluid_day2 + rvcav_dilated:fluid_day2 + micu:rvcav_dilated + 
    micu:fluid_day2 + micu:rvcav_dilated:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3125  -0.7588  -0.5579   0.8569   2.5966  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -2.12203    0.21010 -10.100  < 2e-16 ***
age                            1.63580    0.36200   4.519 6.22e-06 ***
age_over_90                    1.18303    0.21594   5.478 4.29e-08 ***
gender                        -0.03456    0.10531  -0.328 0.742782    
bmi                            0.17436    0.14495   1.203 0.229007    
bmi_missing                    0.38345    0.18973   2.021 0.

In [45]:
model <- update(covariates, ~ . 
        + rvcav_missing + rvcav_dilated
        + fluid_day3_missing + fluid_day3
        + rvcav_dilated * fluid_day3 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day3_missing + 
    fluid_day3 + rvcav_dilated:fluid_day3 + micu:rvcav_dilated + 
    micu:fluid_day3 + micu:rvcav_dilated:fluid_day3, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3442  -0.7565  -0.5612   0.8538   2.3445  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -2.15969    0.21314 -10.133  < 2e-16 ***
age                            1.58583    0.36145   4.387 1.15e-05 ***
age_over_90                    1.16287    0.21610   5.381 7.40e-08 ***
gender                        -0.03495    0.10535  -0.332 0.740069    
bmi                            0.18880    0.16930   1.115 0.264793    
bmi_missing                    0.41212    0.18990   2.170 0.

## Unadjusted

In [46]:
model <- glm(outcome ~ 
        + rvcav_missing + rvcav_dilated
        + fluid_day1_missing + fluid_day1
        + rvcav_dilated * fluid_day1 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ +rvcav_missing + rvcav_dilated + fluid_day1_missing + 
    fluid_day1 + rvcav_dilated * fluid_day1 * micu, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.3722  -0.8168  -0.7319   1.3095   2.0078  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -1.23397    0.13659  -9.034   <2e-16 ***
rvcav_missing                  0.28025    0.14339   1.954   0.0506 .  
rvcav_dilated                 -0.37780    0.39952  -0.946   0.3443    
fluid_day1_missing            -0.17680    0.22129  -0.799   0.4243    
fluid_day1                     0.17099    0.12595   1.358   0.1746    
micu                           0.25573    0.14640   1.747   0.0807 .  
rvcav_dilated:fluid_day1       0.02499    0.45790   0.055   0.9565    
rvcav_dilated:micu             0.60710    0.41811   1.452   0.1465    
fluid_day1:micu                0.12938    0.

In [47]:
model <- glm(outcome ~ 
        + rvcav_missing + rvcav_dilated
        + fluid_day2_missing + fluid_day2
        + rvcav_dilated * fluid_day2 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ +rvcav_missing + rvcav_dilated + fluid_day2_missing + 
    fluid_day2 + rvcav_dilated * fluid_day2 * micu, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2685  -0.8221  -0.7179   1.3244   2.1897  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -1.25175    0.13873  -9.023   <2e-16 ***
rvcav_missing                  0.26811    0.14375   1.865   0.0622 .  
rvcav_dilated                 -0.47510    0.43166  -1.101   0.2711    
fluid_day2_missing            -0.04226    0.14378  -0.294   0.7688    
fluid_day2                     0.24610    0.13665   1.801   0.0717 .  
micu                           0.26874    0.14843   1.811   0.0702 .  
rvcav_dilated:fluid_day2       0.27216    0.45091   0.604   0.5461    
rvcav_dilated:micu             0.70868    0.44897   1.578   0.1145    
fluid_day2:micu                0.12093    0.

In [48]:
model <- glm(outcome ~ 
        + rvcav_missing + rvcav_dilated
        + fluid_day3_missing + fluid_day3
        + rvcav_dilated * fluid_day3 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ +rvcav_missing + rvcav_dilated + fluid_day3_missing + 
    fluid_day3 + rvcav_dilated * fluid_day3 * micu, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2557  -0.8087  -0.7322   1.3028   1.9857  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -1.21300    0.13727  -8.837   <2e-16 ***
rvcav_missing                  0.29128    0.14372   2.027   0.0427 *  
rvcav_dilated                 -0.44156    0.41700  -1.059   0.2896    
fluid_day3_missing            -0.08352    0.10733  -0.778   0.4365    
fluid_day3                     0.09568    0.14732   0.649   0.5160    
micu                           0.23764    0.14594   1.628   0.1034    
rvcav_dilated:fluid_day3       0.32704    0.50458   0.648   0.5169    
rvcav_dilated:micu             0.71178    0.43450   1.638   0.1014    
fluid_day3:micu                0.32857    0.

## Model selection

In [49]:
model <- update(covariates, ~ .
        + rvcav_missing + rvcav_dilated
        + fluid_day1_missing + fluid_day1
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * rvcav_dilated * fluid_day1
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * rvcav_dilated * fluid_day1)
summary(select2)

Start:  AIC=2273.53
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    rvcav_missing + rvcav_dilated + fluid_day1_missing + fluid_day1

                           Df Deviance    AIC
+ micu:fluid_day1           1   2236.2 2272.2
+ rvcav_dilated:fluid_day1  1   2236.6 2272.6
<none>                          2239.5 2273.5
+ micu:rvcav_dilated        1   2237.9 2273.9

Step:  AIC=2272.24
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    rvcav_missing + rvcav_dilated + fluid_day1_missing + fluid_day1 + 
    micu:fluid_day1

                           Df Deviance    AIC
+ rvcav_dilated:fluid_day1  1   2234.0 2272.0
<none>                          2236.2 2272.2
+ micu:rvcav_dilated        1   2234.7 2272.7

Step:  AIC=2272.01
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + cr

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2152,2239.530,2273.530
+ micu:fluid_day1,-1,3.286192,2151,2236.244,2272.244
+ rvcav_dilated:fluid_day1,-1,2.231707,2150,2234.012,2272.012



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day1_missing + 
    fluid_day1 + micu:fluid_day1 + rvcav_dilated:fluid_day1, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3143  -0.7580  -0.5573   0.8610   2.3385  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -2.17407    0.20315 -10.702  < 2e-16 ***
age                       1.66172    0.36215   4.588 4.46e-06 ***
age_over_90               1.21810    0.21608   5.637 1.73e-08 ***
gender                   -0.03605    0.10514  -0.343 0.731695    
bmi                       0.16117    0.12553   1.284 0.199165    
bmi_missing               0.34817    0.19034   1.829 0.067378 .  
mech_vent                 0.37525    0.11197   3.351 0.000804 ***
CHF             


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day1_missing + 
    fluid_day1 + micu:fluid_day1 + rvcav_dilated:fluid_day1 + 
    micu:rvcav_dilated + micu:rvcav_dilated:fluid_day1, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3068  -0.7563  -0.5566   0.8612   2.3715  

Coefficients:
                               Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -2.108006   0.208610 -10.105  < 2e-16 ***
age                            1.663051   0.362311   4.590 4.43e-06 ***
age_over_90                    1.214069   0.216175   5.616 1.95e-08 ***
gender                        -0.035949   0.105205  -0.342 0.732574    
bmi                            0.160800   0.124419   1.292 0.196214    
bmi_missing                    0.346196   0.190295   1

In [50]:
model <- update(covariates, ~ .
        + rvcav_missing + rvcav_dilated
        + fluid_day2_missing + fluid_day2
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * rvcav_dilated * fluid_day2
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * rvcav_dilated * fluid_day2)
summary(select2)

Start:  AIC=2272.92
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    rvcav_missing + rvcav_dilated + fluid_day2_missing + fluid_day2

                           Df Deviance    AIC
+ rvcav_dilated:fluid_day2  1   2235.7 2271.7
+ micu:fluid_day2           1   2236.6 2272.6
<none>                          2238.9 2272.9
+ micu:rvcav_dilated        1   2237.3 2273.3

Step:  AIC=2271.74
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    rvcav_missing + rvcav_dilated + fluid_day2_missing + fluid_day2 + 
    rvcav_dilated:fluid_day2

                     Df Deviance    AIC
<none>                    2235.7 2271.7
+ micu:rvcav_dilated  1   2233.8 2271.8
+ micu:fluid_day2     1   2234.0 2272.0


Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2152,2238.920,2272.920
+ rvcav_dilated:fluid_day2,-1,3.184631,2151,2235.736,2271.736



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day2_missing + 
    fluid_day2 + rvcav_dilated:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2872  -0.7565  -0.5586   0.8734   2.3651  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -2.22153    0.20494 -10.840  < 2e-16 ***
age                       1.61878    0.36149   4.478 7.53e-06 ***
age_over_90               1.18078    0.21568   5.475 4.38e-08 ***
gender                   -0.03498    0.10507  -0.333 0.739205    
bmi                       0.17430    0.14642   1.190 0.233902    
bmi_missing               0.38170    0.18986   2.010 0.044387 *  
mech_vent                 0.38582    0.11294   3.416 0.000635 ***
CHF                       0.02734 


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day2_missing + 
    fluid_day2 + rvcav_dilated:fluid_day2 + micu:rvcav_dilated + 
    micu:fluid_day2 + micu:rvcav_dilated:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3125  -0.7588  -0.5579   0.8569   2.5966  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -2.12203    0.21010 -10.100  < 2e-16 ***
age                            1.63580    0.36200   4.519 6.22e-06 ***
age_over_90                    1.18303    0.21594   5.478 4.29e-08 ***
gender                        -0.03456    0.10531  -0.328 0.742782    
bmi                            0.17436    0.14495   1.203 0.229007    
bmi_missing                    0.38345    0.18973   2.021 0.

In [51]:
model <- update(covariates, ~ .
        + rvcav_missing + rvcav_dilated
        + fluid_day3_missing + fluid_day3
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * rvcav_dilated * fluid_day3
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * rvcav_dilated * fluid_day3)
summary(select2)

Start:  AIC=2270.53
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    rvcav_missing + rvcav_dilated + fluid_day3_missing + fluid_day3

                           Df Deviance    AIC
+ micu:fluid_day3           1   2232.0 2268.0
+ rvcav_dilated:fluid_day3  1   2234.3 2270.3
<none>                          2236.5 2270.5
+ micu:rvcav_dilated        1   2234.9 2270.9

Step:  AIC=2267.99
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    rvcav_missing + rvcav_dilated + fluid_day3_missing + fluid_day3 + 
    micu:fluid_day3

                           Df Deviance    AIC
<none>                          2232.0 2268.0
+ rvcav_dilated:fluid_day3  1   2230.4 2268.4
+ micu:rvcav_dilated        1   2230.5 2268.5


Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2152,2236.533,2270.533
+ micu:fluid_day3,-1,4.545434,2151,2231.988,2267.988



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day3_missing + 
    fluid_day3 + micu:fluid_day3, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3683  -0.7570  -0.5613   0.8494   2.3152  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.21899    0.20755 -10.691  < 2e-16 ***
age                 1.56976    0.36099   4.348 1.37e-05 ***
age_over_90         1.16763    0.21598   5.406 6.44e-08 ***
gender             -0.03738    0.10518  -0.355 0.722291    
bmi                 0.19005    0.17202   1.105 0.269250    
bmi_missing         0.41474    0.19011   2.182 0.029144 *  
mech_vent           0.40447    0.11475   3.525 0.000424 ***
CHF                 0.04191    0.11388   0.368 0.712864    
mdrd                0.02680    


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day3_missing + 
    fluid_day3 + micu:fluid_day3 + micu:rvcav_dilated + rvcav_dilated:fluid_day3 + 
    micu:rvcav_dilated:fluid_day3, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3442  -0.7565  -0.5612   0.8538   2.3445  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -2.15969    0.21314 -10.133  < 2e-16 ***
age                            1.58583    0.36145   4.387 1.15e-05 ***
age_over_90                    1.16287    0.21610   5.381 7.40e-08 ***
gender                        -0.03495    0.10535  -0.332 0.740069    
bmi                            0.18880    0.16930   1.115 0.264793    
bmi_missing                    0.41212    0.18990   2.170 0.

# All echo variables

## Model selection

In [53]:
model <- update(covariates, ~ . + 
    + fluid_day1_missing + lv_missing + plm_htn_missing + rvcav_missing
    + micu * fluid_day1 * lv_hyper 
    + micu * fluid_day1 * lv_mild
    + micu * fluid_day1 * lv_depressed
    + micu * fluid_day1 * lv_severe
    + micu * fluid_day1 * plm_htn_abnormal
    + micu * fluid_day1 * rvcav_dilated
    )
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + fluid_day1_missing + lv_missing + plm_htn_missing + 
    rvcav_missing + fluid_day1 + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + plm_htn_abnormal + rvcav_dilated + micu:fluid_day1 + 
    micu:lv_hyper + fluid_day1:lv_hyper + micu:lv_mild + fluid_day1:lv_mild + 
    micu:lv_depressed + fluid_day1:lv_depressed + micu:lv_severe + 
    fluid_day1:lv_severe + micu:plm_htn_abnormal + fluid_day1:plm_htn_abnormal + 
    micu:rvcav_dilated + fluid_day1:rvcav_dilated + micu:fluid_day1:lv_hyper + 
    micu:fluid_day1:lv_mild + micu:fluid_day1:lv_depressed + 
    micu:fluid_day1:lv_severe + micu:fluid_day1:plm_htn_abnormal + 
    micu:fluid_day1:rvcav_dilated, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3826  -0.7529  -0.5314   0.8364   2.4472  

Coeffi

In [59]:
model <- update(covariates, ~ . + 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe
        + plm_htn_missing + plm_htn_abnormal
        + rvcav_missing + rvcav_dilated 
        + fluid_day1_missing + fluid_day1
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~. + 
    + micu * fluid_day1 * lv_hyper 
    + micu * fluid_day1 * lv_mild
    + micu * fluid_day1 * lv_depressed
    + micu * fluid_day1 * lv_severe
    + micu * fluid_day1 * plm_htn_abnormal
    + micu * fluid_day1 * rvcav_dilated
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * fluid_day1 * lv_mild)
summary(select2)

Start:  AIC=2260.43
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    plm_htn_missing + plm_htn_abnormal + rvcav_missing + rvcav_dilated + 
    fluid_day1_missing + fluid_day1

                              Df Deviance    AIC
+ rvcav_dilated:fluid_day1     1   2208.9 2258.9
+ micu:fluid_day1              1   2209.0 2259.0
+ plm_htn_abnormal:fluid_day1  1   2210.3 2260.3
<none>                             2212.4 2260.4
+ micu:rvcav_dilated           1   2210.7 2260.7
+ lv_severe:fluid_day1         1   2210.9 2260.9
+ micu:lv_severe               1   2212.0 2262.0
+ lv_depressed:fluid_day1      1   2212.0 2262.0
+ micu:plm_htn_abnormal        1   2212.2 2262.2
+ micu:lv_hyper                1   2212.3 2262.3
+ lv_hyper:fluid_day1          1   2212.4 2262.4
+ micu:lv_mild                 1   2212.4 2262.4
+ micu:lv_depressed          

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2145,2212.429,2260.429
+ rvcav_dilated:fluid_day1,-1,3.487142,2144,2208.942,2258.942
+ micu:fluid_day1,-1,2.626685,2143,2206.316,2258.316



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + plm_htn_missing + plm_htn_abnormal + rvcav_missing + 
    rvcav_dilated + fluid_day1_missing + fluid_day1 + rvcav_dilated:fluid_day1 + 
    micu:fluid_day1, family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3780  -0.7539  -0.5391   0.8317   2.4974  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -2.35434    0.24121  -9.760  < 2e-16 ***
age                       1.48460    0.37128   3.999 6.37e-05 ***
age_over_90               1.09505    0.22167   4.940 7.82e-07 ***
gender                   -0.08022    0.10717  -0.749 0.454123    
bmi                       0.17790    0.14625   1.216 0.223840    
bmi_missing               0.39306    0.19345 


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + plm_htn_missing + plm_htn_abnormal + rvcav_missing + 
    rvcav_dilated + fluid_day1_missing + fluid_day1 + rvcav_dilated:fluid_day1 + 
    micu:fluid_day1 + micu:lv_mild + lv_mild:fluid_day1 + micu:lv_mild:fluid_day1, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3925  -0.7534  -0.5346   0.8279   2.4489  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -2.361613   0.242894  -9.723  < 2e-16 ***
age                       1.521379   0.372608   4.083 4.44e-05 ***
age_over_90               1.100226   0.222229   4.951 7.39e-07 ***
gender                   -0.080187   0.107383  -0.747  0.45522    
bmi                       0.186385   0.

In [60]:
model <- update(covariates, ~ . + 
    + fluid_day1_missing + lv_missing + plm_htn_missing + rvcav_missing
    + micu * fluid_day2 * lv_hyper 
    + micu * fluid_day2 * lv_mild
    + micu * fluid_day2 * lv_depressed
    + micu * fluid_day2 * lv_severe
    + micu * fluid_day2 * plm_htn_abnormal
    + micu * fluid_day2 * rvcav_dilated
    )
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + fluid_day1_missing + lv_missing + plm_htn_missing + 
    rvcav_missing + fluid_day2 + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + plm_htn_abnormal + rvcav_dilated + micu:fluid_day2 + 
    micu:lv_hyper + fluid_day2:lv_hyper + micu:lv_mild + fluid_day2:lv_mild + 
    micu:lv_depressed + fluid_day2:lv_depressed + micu:lv_severe + 
    fluid_day2:lv_severe + micu:plm_htn_abnormal + fluid_day2:plm_htn_abnormal + 
    micu:rvcav_dilated + fluid_day2:rvcav_dilated + micu:fluid_day2:lv_hyper + 
    micu:fluid_day2:lv_mild + micu:fluid_day2:lv_depressed + 
    micu:fluid_day2:lv_severe + micu:fluid_day2:plm_htn_abnormal + 
    micu:fluid_day2:rvcav_dilated, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3945  -0.7499  -0.5287   0.8075   2.4203  

Coeffi

In [61]:
model <- update(covariates, ~ . + 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe
        + plm_htn_missing + plm_htn_abnormal
        + rvcav_missing + rvcav_dilated 
        + fluid_day2_missing + fluid_day2
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~. + 
    + micu * fluid_day2 * lv_hyper 
    + micu * fluid_day2 * lv_mild
    + micu * fluid_day2 * lv_depressed
    + micu * fluid_day2 * lv_severe
    + micu * fluid_day2 * plm_htn_abnormal
    + micu * fluid_day2 * rvcav_dilated
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * fluid_day2 * lv_mild)
summary(select2)

Start:  AIC=2259.35
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    plm_htn_missing + plm_htn_abnormal + rvcav_missing + rvcav_dilated + 
    fluid_day2_missing + fluid_day2

                              Df Deviance    AIC
+ rvcav_dilated:fluid_day2     1   2207.3 2257.3
+ lv_severe:fluid_day2         1   2208.2 2258.2
+ lv_mild:fluid_day2           1   2208.8 2258.8
+ micu:fluid_day2              1   2208.8 2258.8
+ plm_htn_abnormal:fluid_day2  1   2209.1 2259.1
<none>                             2211.3 2259.3
+ lv_depressed:fluid_day2      1   2209.5 2259.5
+ micu:rvcav_dilated           1   2209.6 2259.6
+ micu:lv_severe               1   2210.8 2260.8
+ micu:lv_hyper                1   2211.1 2261.1
+ lv_hyper:fluid_day2          1   2211.2 2261.2
+ micu:plm_htn_abnormal        1   2211.2 2261.2
+ micu:lv_mild               

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2145,2211.346,2259.346
+ rvcav_dilated:fluid_day2,-1,4.052163,2144,2207.294,2257.294
+ lv_mild:fluid_day2,-1,2.477494,2143,2204.816,2256.816
+ micu:rvcav_dilated,-1,2.233897,2142,2202.582,2256.582



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + plm_htn_missing + plm_htn_abnormal + rvcav_missing + 
    rvcav_dilated + fluid_day2_missing + fluid_day2 + rvcav_dilated:fluid_day2 + 
    lv_mild:fluid_day2 + micu:rvcav_dilated, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2940  -0.7540  -0.5401   0.8506   2.3988  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -2.33382    0.24900  -9.373  < 2e-16 ***
age                       1.44430    0.37103   3.893 9.92e-05 ***
age_over_90               1.05944    0.22188   4.775 1.80e-06 ***
gender                   -0.08016    0.10734  -0.747 0.455191    
bmi                       0.19692    0.18258   1.079 0.280794    
bmi_missing     


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + plm_htn_missing + plm_htn_abnormal + rvcav_missing + 
    rvcav_dilated + fluid_day2_missing + fluid_day2 + rvcav_dilated:fluid_day2 + 
    lv_mild:fluid_day2 + micu:rvcav_dilated + micu:fluid_day2 + 
    micu:lv_mild + micu:lv_mild:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3281  -0.7543  -0.5312   0.8276   2.3691  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -2.32364    0.25002  -9.294  < 2e-16 ***
age                       1.49540    0.37284   4.011 6.05e-05 ***
age_over_90               1.06413    0.22288   4.774 1.80e-06 ***
gender                   -0.08537    0.10767  -0.793 0.427813    
bmi               

In [57]:
model <- update(covariates, ~ . + 
    + fluid_day1_missing + lv_missing + plm_htn_missing + rvcav_missing
    + micu * fluid_day3 * lv_hyper 
    + micu * fluid_day3 * lv_mild
    + micu * fluid_day3 * lv_depressed
    + micu * fluid_day3 * lv_severe
    + micu * fluid_day3 * plm_htn_abnormal
    + micu * fluid_day3 * rvcav_dilated
    )
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + fluid_day1_missing + lv_missing + plm_htn_missing + 
    rvcav_missing + fluid_day3 + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + plm_htn_abnormal + rvcav_dilated + micu:fluid_day3 + 
    micu:lv_hyper + fluid_day3:lv_hyper + micu:lv_mild + fluid_day3:lv_mild + 
    micu:lv_depressed + fluid_day3:lv_depressed + micu:lv_severe + 
    fluid_day3:lv_severe + micu:plm_htn_abnormal + fluid_day3:plm_htn_abnormal + 
    micu:rvcav_dilated + fluid_day3:rvcav_dilated + micu:fluid_day3:lv_hyper + 
    micu:fluid_day3:lv_mild + micu:fluid_day3:lv_depressed + 
    micu:fluid_day3:lv_severe + micu:fluid_day3:plm_htn_abnormal + 
    micu:fluid_day3:rvcav_dilated, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4237  -0.7487  -0.5332   0.8101   2.5212  

Coeffi

In [62]:
model <- update(covariates, ~ . + 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe
        + plm_htn_missing + plm_htn_abnormal
        + rvcav_missing + rvcav_dilated 
        + fluid_day3_missing + fluid_day3
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~. + 
    + micu * fluid_day3 * lv_hyper 
    + micu * fluid_day3 * lv_mild
    + micu * fluid_day3 * lv_depressed
    + micu * fluid_day3 * lv_severe
    + micu * fluid_day3 * plm_htn_abnormal
    + micu * fluid_day3 * rvcav_dilated
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * fluid_day3 * lv_mild)
summary(select2)

Start:  AIC=2257.15
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    plm_htn_missing + plm_htn_abnormal + rvcav_missing + rvcav_dilated + 
    fluid_day3_missing + fluid_day3

                              Df Deviance    AIC
+ micu:fluid_day3              1   2204.3 2254.3
+ lv_mild:fluid_day3           1   2205.8 2255.8
+ rvcav_dilated:fluid_day3     1   2206.4 2256.4
+ lv_depressed:fluid_day3      1   2206.4 2256.4
+ plm_htn_abnormal:fluid_day3  1   2206.6 2256.6
<none>                             2209.2 2257.2
+ micu:rvcav_dilated           1   2207.4 2257.4
+ micu:lv_severe               1   2208.7 2258.7
+ lv_severe:fluid_day3         1   2208.8 2258.8
+ micu:plm_htn_abnormal        1   2209.0 2259.0
+ micu:lv_hyper                1   2209.0 2259.0
+ micu:lv_mild                 1   2209.1 2259.1
+ micu:lv_depressed          

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2145,2209.153,2257.153
+ micu:fluid_day3,-1,4.897568,2144,2204.255,2254.255
+ lv_mild:fluid_day3,-1,3.549096,2143,2200.706,2252.706
+ lv_depressed:fluid_day3,-1,2.240157,2142,2198.466,2252.466



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + plm_htn_missing + plm_htn_abnormal + rvcav_missing + 
    rvcav_dilated + fluid_day3_missing + fluid_day3 + micu:fluid_day3 + 
    lv_mild:fluid_day3 + lv_depressed:fluid_day3, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3738  -0.7487  -0.5400   0.8341   2.4759  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.43771    0.24712  -9.865  < 2e-16 ***
age                      1.43206    0.37193   3.850 0.000118 ***
age_over_90              1.07794    0.22276   4.839 1.31e-06 ***
gender                  -0.08560    0.10751  -0.796 0.425894    
bmi                      0.21739    0.22553   0.964 0.335084    
bmi_missing              0


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + plm_htn_missing + plm_htn_abnormal + rvcav_missing + 
    rvcav_dilated + fluid_day3_missing + fluid_day3 + micu:fluid_day3 + 
    lv_mild:fluid_day3 + lv_depressed:fluid_day3 + micu:lv_mild + 
    micu:lv_mild:fluid_day3, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3807  -0.7484  -0.5354   0.8251   2.5085  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.45091    0.24870  -9.855  < 2e-16 ***
age                      1.46550    0.37321   3.927 8.61e-05 ***
age_over_90              1.08859    0.22329   4.875 1.09e-06 ***
gender                  -0.07981    0.10775  -0.741 0.458850    
bmi                      0.22759    0.25144  